In [25]:
# how to create tools
# how to use builtin tools like SerpAPI, Wikipedia, etc. and toolkits
# how to use chat models to call tools
# how to pass tool outputs to chat models


In [26]:
#create tool using @tool decorator (defining a custom tool)
from langchain.tools import tool
@tool
async def division(a:int, b:int) -> float:
    """Divides two numbers."""
    return a / b

In [27]:
print(division.name)
print(division.description)
print(division.args)

division
Divides two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [28]:
from typing import Annotated,List

@tool
def multiply_by_max(
    a:Annotated[int,"A value"],
    b:Annotated[List[int],"B list value"]
)->int:
    """multiply by maximum"""
    return a*max(b)

In [29]:
### Structured Tool
from langchain_core.tools import StructuredTool
def multiply(a:int, b:int) -> int:
    """Multiply two numbers."""
    return a * b
async def amultiply(a:int, b:int) -> int:
    """Asynchronously multiply two numbers."""
    return a * b

# StructuredTool expects the callable under the `func` argument, not `fun`
calculator = StructuredTool.from_function(func=multiply, coroutine=amultiply)
print(calculator.invoke({"a":2,"b":3}))
print(await calculator.ainvoke({"a":3,"b":6}))

6
18


In [30]:
#using inbuilt tool (e.g. wikipedia)
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper=WikipediaAPIWrapper(top_k_results=5,doc_content_chars_max=500)
wiki_tool=WikipediaQueryRun(api_wrapper=api_wrapper)
print(wiki_tool.invoke({"query":"langchain"}))


Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.



Page: Retrieval-augmented generation
Summary: Retrieval-augmented generation (RAG) is a technique that enables large language models (LLMs) to retriev


In [31]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [32]:
#tavily helps in browsing the web
os.environ["TAVILY_API_KEY"]=os.getenv("TAVILY_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")


In [33]:
from langchain_tavily import TavilySearch
tavily_tool=TavilySearch(
    max_results=5,
    topics=["technology","science"]
)

In [34]:
tavily_tool.invoke("What is the recent AI news??")

{'query': 'What is the recent AI news??',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://www.crescendo.ai/news/latest-ai-news-and-updates',
   'title': 'The Latest AI News and AI Breakthroughs that Matter Most: 2025',
   'content': 'Summary: Xiaomi has announced a next-gen AI voice model optimized for in-car and smart home experiences. The model features faster response times, offline',
   'score': 0.99943846,
   'raw_content': None},
  {'url': 'https://www.wsj.com/tech/ai?gaa_at=eafs&gaa_n=AWEtsqd8LHH6MzE2QK0F7OdQv3I-20Nz49id4xEpUEsOzxg3rt8pdGFvzRXl&gaa_ts=694593a7&gaa_sig=jf23knIhqcdMyvD5qX3rhxE0fXp04GQiieWcNifu_t13vCJEZ5OB7HArKRdyRGY8PymtC5INnrgf_lFUsuIbFQ%3D%3D',
   'title': 'Artificial Intelligence - Latest AI News and Analysis - WSJ.com',
   'content': "CEOs to Keep Spending on AI, Despite Spotty Returns · Teneo's annual survey finds 68% of chief executives plan to increase AI spending in 2026. By. Ben Glickman.",
   'score': 0.9981614

In [35]:
# using tools in chat models
tools=[tavily_tool,division,wiki_tool]

In [36]:
##integrating tools with our llm model
from langchain.chat_models import init_chat_model
llm=init_chat_model("qwen/qwen3-32b",model_provider="groq")
llm

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001D9E1760E00>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001D9E12CDBE0>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [37]:
llm_with_tool=llm.bind_tools(tools)

In [38]:
from langchain_core.messages import HumanMessage
query="whatis 2 divided by 3"
messages=[HumanMessage(query)]
response=llm_with_tool.invoke(query)
print(response)


content='' additional_kwargs={'reasoning_content': 'Okay, let\'s see. The user is asking "whatis 2 divided by 3". First, I need to understand what they\'re asking. It looks like a simple division problem. The functions available include a division tool, which takes two integers, a and b, and returns a divided by b. Since the user is asking for 2 divided by 3, I should use the division function here. The parameters are a=2 and b=3. I don\'t need to use any other tools like the search engine or Wikipedia because this is a straightforward arithmetic question. So the correct tool to call is the division function with those values.\n', 'tool_calls': [{'id': 'tfwcbsdrj', 'function': {'arguments': '{"a":2,"b":3}', 'name': 'division'}, 'type': 'function'}]} response_metadata={'token_usage': {'completion_tokens': 161, 'prompt_tokens': 1815, 'total_tokens': 1976, 'completion_time': 0.284882255, 'completion_tokens_details': {'reasoning_tokens': 132}, 'prompt_time': 0.093748052, 'prompt_tokens_det

In [39]:
response.tool_calls

[{'name': 'division',
  'args': {'a': 2, 'b': 3},
  'id': 'tfwcbsdrj',
  'type': 'tool_call'}]

In [57]:
async def execute_tools(response, messages):
    for tool_call in response.tool_calls:
        selected_tool = {
            "division": division,
            "tavily_search": tavily_tool,
            "wikipedia": wiki_tool
        }[tool_call["name"].lower()]

        # Run tool
        result = await selected_tool.ainvoke(tool_call["args"])

        # Wrap result correctly
        tool_message = ToolMessage(
            content=str(result),
            tool_call_id=tool_call["id"]
        )

        messages.append(tool_message)

    return messages


In [58]:
final_response = await llm_with_tool.ainvoke(messages)


NotImplementedError: Unsupported message type: <class 'float'>
For troubleshooting, visit: https://docs.langchain.com/oss/python/langchain/errors/MESSAGE_COERCION_FAILURE 

In [ ]:
llm_with_tool.invoke(messages)

<coroutine object RunnableBindingBase.ainvoke at 0x000001D9E16ECE50>